In [76]:
import pandas as pd

In [77]:
def get_data() -> tuple[pd.DataFrame, list]:
    """
    获得股票历史信息，并计算总市值
    """
    # 显示结果
    df = pd.read_csv(
        "../data/Hist_2023-03-20.csv", parse_dates=['日期'], index_col=0, dtype={"股票代码": object})
    dates = df.index.unique().sort_values().to_list()
    # print(type(dates[0]))
    # dates = [x.strftime("%Y-%m-%d") for x in dates]
    # 获得当前结果集的日期列表
    dates_list = [date.strftime('%Y-%m-%d') for date in dates]
    value = pd.read_csv("../data/总股本.csv", index_col=0, dtype={"代码": object})
    value_dict = value['总股本'].to_dict()
    df['总股本'] = df['股票代码'].apply(lambda x: value_dict.get(x))
    df['总市值'] = df['总股本']*df['收盘']

    return (df, dates_list)


In [78]:
df, _ = get_data()
df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 71152 entries, 2023-03-01 to 2023-03-20
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   开盘      71152 non-null  float64
 1   收盘      71152 non-null  float64
 2   最高      71152 non-null  float64
 3   最低      71152 non-null  float64
 4   成交量     71152 non-null  int64  
 5   成交额     71152 non-null  float64
 6   振幅      71152 non-null  float64
 7   涨跌幅     71152 non-null  float64
 8   涨跌额     71152 non-null  float64
 9   换手率     71152 non-null  float64
 10  股票代码    71152 non-null  object 
 11  板块名称    71152 non-null  object 
 12  总股本     68019 non-null  float64
 13  总市值     68019 non-null  float64
dtypes: float64(11), int64(1), object(2)
memory usage: 10.2+ MB


In [105]:
df.columns

Index(['开盘', '收盘', '最高', '最低', '成交量', '成交额', '振幅', '涨跌幅', '涨跌额', '换手率', '股票代码',
       '板块名称', '总股本', '总市值'],
      dtype='object')

In [79]:
# cur_date="2023-03-01"
# cur_df = df.loc[cur_date]
cur_df = df.copy()
# 按股票名称分组，并统计涨幅大于0和小于0的股票数量
result = cur_df.groupby(['日期', '板块名称'])['涨跌幅'].agg(
    [('涨的数量', lambda x: sum(x > 0)), ('跌的数量', lambda x: sum(x < 0))])
result['涨幅比'] = result['涨的数量']/(result['涨的数量']+result['跌的数量'])*100
result


涨的数量  跌的数量        涨幅比
日期         板块名称                           
2023-03-01 IT服务       120     7  94.488189
           LED         30     7  81.081081
           专业工程        31     3  91.176471
           专业服务        20     6  76.923077
           专业连锁         6     1  85.714286
...                   ...   ...        ...
2023-03-20 食品及饲料添加剂     6    13  31.578947
           餐饮           2     1  66.666667
           饰品          11     5  68.750000
           饲料           2    16  11.111111
           高速公路         5    13  27.777778

[3290 rows x 3 columns]

In [80]:
cur_df = df.copy()
value_df = cur_df.groupby(['日期', "板块名称"]).agg(
    {"涨跌幅": "mean", "总市值": "sum"})
value_df


涨跌幅           总市值
日期         板块名称                            
2023-03-01 IT服务      4.119685  1.235165e+12
           LED       1.122821  2.941001e+11
           专业工程      1.343529  3.725464e+11
           专业服务      0.585556  1.848354e+11
           专业连锁      0.792500  6.418736e+10
...                       ...           ...
2023-03-20 食品及饲料添加剂 -0.666500  1.584158e+11
           餐饮        0.846667  2.143710e+10
           饰品        0.995625  1.220344e+11
           饲料       -1.441667  2.820616e+11
           高速公路     -0.829500  2.353763e+11

[3290 rows x 2 columns]

In [81]:
# cur_df = cur_df[(cur_df['总市值'] >= (start_value)*100_000_000)
#                 & (cur_df['总市值'] <= (end_value)*100_000_000)]
# cur_df

final_df = result.join(value_df, on=["日期","板块名称"])
# final_df.dropna(inplace=True, axis=0)
# 将Salary列格式化为亿元
final_df['总市值亿元'] = final_df['总市值'].apply(
    lambda x: '{:.2f}'.format(x/100000000))
final_df


涨的数量  跌的数量        涨幅比       涨跌幅           总市值     总市值亿元
日期         板块名称                                                             
2023-03-01 IT服务       120     7  94.488189  4.119685  1.235165e+12  12351.65
           LED         30     7  81.081081  1.122821  2.941001e+11   2941.00
           专业工程        31     3  91.176471  1.343529  3.725464e+11   3725.46
           专业服务        20     6  76.923077  0.585556  1.848354e+11   1848.35
           专业连锁         6     1  85.714286  0.792500  6.418736e+10    641.87
...                   ...   ...        ...       ...           ...       ...
2023-03-20 食品及饲料添加剂     6    13  31.578947 -0.666500  1.584158e+11   1584.16
           餐饮           2     1  66.666667  0.846667  2.143710e+10    214.37
           饰品          11     5  68.750000  0.995625  1.220344e+11   1220.34
           饲料           2    16  11.111111 -1.441667  2.820616e+11   2820.62
           高速公路         5    13  27.777778 -0.829500  2.353763e+11   2353.76

[3290 rows x 6 columns]

In [82]:
# 按涨跌幅统计
x = ["跌停", "跌<-5%",  "-3%>-5%",     "-1-3%",    "平盘",
     "<3%",     "3-5%",   "5%-涨停", "涨停"]

db = df.copy()
# db = df.loc['2023-03-01']
bins = [-20, -10, -5, -3, -0.099, 0.099, 3, 5, 10, 20]
# bins = list(range(-11, 12))
cuts = pd.cut(db['涨跌幅'], bins=bins)

x = ["跌停", "跌<-5%",  "-5<-3%",     "-1<3%",    "平盘",
     "1<3%",     "3-5%",   "5%-涨停", "涨停"]
cuts.categories =x
cuts

日期
2023-03-01      (0.099, 3.0]
2023-03-02      (-5.0, -3.0]
2023-03-03    (-3.0, -0.099]
2023-03-06      (0.099, 3.0]
2023-03-07    (-3.0, -0.099]
                   ...      
2023-03-14    (-3.0, -0.099]
2023-03-15       (5.0, 10.0]
2023-03-16      (0.099, 3.0]
2023-03-17    (-3.0, -0.099]
2023-03-20    (-3.0, -0.099]
Name: 涨跌幅, Length: 71152, dtype: category
Categories (9, interval[float64, right]): [(-20.0, -10.0] < (-10.0, -5.0] < (-5.0, -3.0] < (-3.0, -0.099] ... (0.099, 3.0] < (3.0, 5.0] < (5.0, 10.0] < (10.0, 20.0]]

In [83]:
pct_chg_list = db.groupby(["日期","板块名称", cuts])['涨跌幅'].count()
pct_chg_list


日期          板块名称  涨跌幅            
2023-03-01  IT服务  (-20.0, -10.0]     0
                  (-10.0, -5.0]      0
                  (-5.0, -3.0]       0
                  (-3.0, -0.099]     6
                  (-0.099, 0.099]    1
                                    ..
2023-03-20  高速公路  (-0.099, 0.099]    2
                  (0.099, 3.0]       5
                  (3.0, 5.0]         0
                  (5.0, 10.0]        0
                  (10.0, 20.0]       0
Name: 涨跌幅, Length: 29610, dtype: int64

In [84]:
pct_chg_list.index



MultiIndex([('2023-03-01', 'IT服务',  (-20.0, -10.0]),
            ('2023-03-01', 'IT服务',   (-10.0, -5.0]),
            ('2023-03-01', 'IT服务',    (-5.0, -3.0]),
            ('2023-03-01', 'IT服务',  (-3.0, -0.099]),
            ('2023-03-01', 'IT服务', (-0.099, 0.099]),
            ('2023-03-01', 'IT服务',    (0.099, 3.0]),
            ('2023-03-01', 'IT服务',      (3.0, 5.0]),
            ('2023-03-01', 'IT服务',     (5.0, 10.0]),
            ('2023-03-01', 'IT服务',    (10.0, 20.0]),
            ('2023-03-01',  'LED',  (-20.0, -10.0]),
            ...
            ('2023-03-20',   '饲料',    (10.0, 20.0]),
            ('2023-03-20', '高速公路',  (-20.0, -10.0]),
            ('2023-03-20', '高速公路',   (-10.0, -5.0]),
            ('2023-03-20', '高速公路',    (-5.0, -3.0]),
            ('2023-03-20', '高速公路',  (-3.0, -0.099]),
            ('2023-03-20', '高速公路', (-0.099, 0.099]),
            ('2023-03-20', '高速公路',    (0.099, 3.0]),
            ('2023-03-20', '高速公路',      (3.0, 5.0]),
            ('2023-03-20', '高速

In [118]:
x = ["跌停", "跌<-5%",  "-5<-3%",     "-1<3%",    "平盘",
     "1<3%",     "3-5%",   "5%-涨停", "涨停"]
db = pd.DataFrame(pct_chg_list)
db

涨跌幅
日期         板块名称 涨跌幅                 
2023-03-01 IT服务 (-20.0, -10.0]     0
                (-10.0, -5.0]      0
                (-5.0, -3.0]       0
                (-3.0, -0.099]     6
                (-0.099, 0.099]    1
...                              ...
2023-03-20 高速公路 (-0.099, 0.099]    2
                (0.099, 3.0]       5
                (3.0, 5.0]         0
                (5.0, 10.0]        0
                (10.0, 20.0]       0

[29610 rows x 1 columns]

In [ ]:
db.reset_index(inplace=True)
db


In [102]:
db.columns

Index(['日期', '板块名称', '涨跌幅'], dtype='object')

In [119]:
ab=pct_chg_list.unstack()
ab

涨跌幅                  (-20.0, -10.0]  (-10.0, -5.0]  (-5.0, -3.0]  \
日期         板块名称                                                    
2023-03-01 IT服务                   0              0             0   
           LED                    0              0             0   
           专业工程                   0              0             0   
           专业服务                   0              0             0   
           专业连锁                   0              1             0   
...                             ...            ...           ...   
2023-03-20 食品及饲料添加剂               0              1             0   
           餐饮                     0              0             0   
           饰品                     0              1             1   
           饲料                     0              0             1   
           高速公路                   0              0             1   

涨跌幅                  (-3.0, -0.099]  (-0.099, 0.099]  (0.099, 3.0]  \
日期         板块名称                                                      
2023-03-01 IT服务                   6                1            41   
           LED                    5                5            27   
           专业工程                   3                1            28   
           专业服务                   6                4            14   
           专业连锁                   0                1             5   
...                             ...              ...           ...   
2023-03-20 食品及饲料添加剂              12                1             5   
           餐饮                     1                0             1   
           饰品                     3                0             6   
           饲料                    15                0             2   
           高速公路                  12                2             5   

涨跌幅                  (3.0, 5.0]  (5.0, 10.0]  (10.0, 20.0]  
日期         板块名称                                             
2023-03-01 IT服务              45           29             4  
           LED                1            1             0  
           专业工程               1            1             0  
           专业服务               3            0             0  
           专业连锁               0            1             0  
...                         ...          ...           ...  
2023-03-20 食品及饲料添加剂           1            0             0  
           餐饮                 1            0             0  
           饰品                 3            2             0  
           饲料                 0            0             0  
           高速公路               0            0             0  

[3290 rows x 9 columns]

In [85]:
db = pd.DataFrame(pct_chg_list)
# db.reset_index(inplace=True,level=[0, 1])
db = db.unstack()
db


涨跌幅                                            \
涨跌幅                 (-20.0, -10.0] (-10.0, -5.0] (-5.0, -3.0] (-3.0, -0.099]   
日期         板块名称                                                                
2023-03-01 IT服务                  0             0            0              6   
           LED                   0             0            0              5   
           专业工程                  0             0            0              3   
           专业服务                  0             0            0              6   
           专业连锁                  0             1            0              0   
...                            ...           ...          ...            ...   
2023-03-20 食品及饲料添加剂              0             1            0             12   
           餐饮                    0             0            0              1   
           饰品                    0             1            1              3   
           饲料                    0             0            1             15   
           高速公路                  0             0            1             12   

                                                                         \
涨跌幅                 (-0.099, 0.099] (0.099, 3.0] (3.0, 5.0] (5.0, 10.0]   
日期         板块名称                                                           
2023-03-01 IT服务                   1           41         45          29   
           LED                    5           27          1           1   
           专业工程                   1           28          1           1   
           专业服务                   4           14          3           0   
           专业连锁                   1            5          0           1   
...                             ...          ...        ...         ...   
2023-03-20 食品及饲料添加剂               1            5          1           0   
           餐饮                     0            1          1           0   
           饰品                     0            6          3           2   
           饲料                     0            2          0           0   
           高速公路                   2            5          0           0   

                                  
涨跌幅                 (10.0, 20.0]  
日期         板块名称                   
2023-03-01 IT服务                4  
           LED                 0  
           专业工程                0  
           专业服务                0  
           专业连锁                0  
...                          ...  
2023-03-20 食品及饲料添加剂            0  
           餐饮                  0  
           饰品                  0  
           饲料                  0  
           高速公路                0  

[3290 rows x 9 columns]

In [86]:
result = pd.merge(cur_df, db, on=["日期",'板块名称'])
result.head()


/var/folders/my/x1pz96mj4_l31xr74gb254fh0000gn/T/ipykernel_93230/2870815350.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  result = pd.merge(cur_df, db, on=["日期",'板块名称'])


,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率,...,总市值,"(涨跌幅, (-20.0, -10.0])","(涨跌幅, (-10.0, -5.0])","(涨跌幅, (-5.0, -3.0])","(涨跌幅, (-3.0, -0.099])","(涨跌幅, (-0.099, 0.099])","(涨跌幅, (0.099, 3.0])","(涨跌幅, (3.0, 5.0])","(涨跌幅, (5.0, 10.0])","(涨跌幅, (10.0, 20.0])"
日期,,,,,,,,,,,,,,,,,,,,,
2023-03-01,22.55,22.42,22.83,22.18,98735,2.219585e+08,2.90,0.18,0.04,3.03,...,1.049298e+10,0,0,2,5,2,4,0,0,0
2023-03-01,4.43,4.45,4.48,4.40,99091,4.394215e+07,1.81,0.68,0.03,0.87,...,5.049222e+09,0,0,2,5,2,4,0,0,0
2023-03-01,12.60,12.48,12.60,12.40,145654,1.820175e+08,1.59,-0.95,-0.12,1.30,...,1.398221e+10,0,0,2,5,2,4,0,0,0
2023-03-01,8.37,8.36,8.38,8.32,24992,2.087038e+07,0.72,0.00,0.00,0.61,...,3.432984e+09,0,0,2,5,2,4,0,0,0
2023-03-01,21.79,21.84,22.08,21.66,15582,3.401650e+07,1.92,-0.27,-0.06,1.88,...,3.626620e+09,0,0,2,5,2,4,0,0,0


In [87]:
result.describe()

,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率,...,总市值,"(涨跌幅, (-20.0, -10.0])","(涨跌幅, (-10.0, -5.0])","(涨跌幅, (-5.0, -3.0])","(涨跌幅, (-3.0, -0.099])","(涨跌幅, (-0.099, 0.099])","(涨跌幅, (0.099, 3.0])","(涨跌幅, (3.0, 5.0])","(涨跌幅, (5.0, 10.0])","(涨跌幅, (10.0, 20.0])"
count,71152.000000,71152.000000,71152.000000,71152.000000,7.115200e+04,7.115200e+04,71152.000000,71152.000000,71152.000000,71152.000000,...,6.801900e+04,71152.000000,71152.000000,71152.000000,71152.000000,71152.000000,71152.000000,71152.000000,71152.000000,71152.000000
mean,23.987596,23.931781,24.420136,23.545071,1.380822e+05,1.731475e+08,3.144107,-0.199287,-0.067575,2.588709,...,1.668627e+10,0.039282,0.636272,3.249930,22.474266,2.293288,15.466607,2.033646,1.111536,0.257393
std,43.759259,43.608046,44.504867,42.937664,3.469410e+05,3.758602e+08,2.032002,2.493202,1.351022,4.000849,...,5.824850e+10,0.208975,1.663168,6.800186,26.628040,3.226858,19.765015,4.640940,2.941410,0.779872
min,0.610000,0.610000,0.610000,0.610000,3.000000e+01,1.645800e+04,0.000000,-51.570000,-54.990000,0.000000,...,3.823060e+08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.870000,6.850000,6.960000,6.770000,1.921800e+04,3.078630e+07,1.890000,-1.560000,-0.210000,0.740000,...,3.378202e+09,0.000000,0.000000,0.000000,6.000000,0.000000,3.000000,0.000000,0.000000,0.000000
50%,13.230000,13.210000,13.430000,13.040000,5.311400e+04,6.827925e+07,2.670000,-0.320000,-0.030000,1.370000,...,5.827296e+09,0.000000,0.000000,1.000000,13.000000,1.000000,9.000000,1.000000,0.000000,0.000000
75%,25.880000,25.810000,26.320000,25.420000,1.361428e+05,1.659724e+08,3.790000,0.820000,0.090000,2.730000,...,1.309167e+10,0.000000,1.000000,3.000000,27.000000,3.000000,20.000000,2.000000,1.000000,0.000000
max,1839.770000,1838.530000,1848.000000,1821.100000,1.670116e+07,1.192481e+10,81.760000,175.390000,122.280000,74.970000,...,2.309557e+12,2.000000,15.000000,55.000000,161.000000,25.000000,145.000000,45.000000,29.000000,7.000000


In [88]:
result['开盘']


日期
2023-03-01    22.55
2023-03-01     4.43
2023-03-01    12.60
2023-03-01     8.37
2023-03-01    21.79
              ...  
2023-03-20     3.58
2023-03-20     3.40
2023-03-20     2.17
2023-03-20    19.69
2023-03-20     4.60
Name: 开盘, Length: 71152, dtype: float64

In [89]:
result.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 71152 entries, 2023-03-01 to 2023-03-20
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   开盘                      71152 non-null  float64
 1   收盘                      71152 non-null  float64
 2   最高                      71152 non-null  float64
 3   最低                      71152 non-null  float64
 4   成交量                     71152 non-null  int64  
 5   成交额                     71152 non-null  float64
 6   振幅                      71152 non-null  float64
 7   涨跌幅                     71152 non-null  float64
 8   涨跌额                     71152 non-null  float64
 9   换手率                     71152 non-null  float64
 10  股票代码                    71152 non-null  object 
 11  板块名称                    71152 non-null  object 
 12  总股本                     68019 non-null  float64
 13  总市值                     68019 non-null  float64
 14  (涨跌幅, (-20.0, -10.0])

In [90]:
result["涨跌幅"]


日期
2023-03-01    0.18
2023-03-01    0.68
2023-03-01   -0.95
2023-03-01    0.00
2023-03-01   -0.27
              ... 
2023-03-20    9.97
2023-03-20    0.29
2023-03-20    2.80
2023-03-20    6.85
2023-03-20    1.73
Name: 涨跌幅, Length: 71152, dtype: float64

In [91]:
result.to_csv("result_20230322.csv")

In [92]:
x = ["跌停", "跌<-5%",  "-5<-3%",     "-1<3%",    "平盘",
     "1<3%",     "3-5%",   "5%-涨停", "涨停"]


In [93]:
result.rename(columns={"(涨跌幅, (-20.0, -10.0])":"跌停"},inplace=True)

In [94]:
result.columns[14:] 


Index([ ('涨跌幅', (-20.0, -10.0]),   ('涨跌幅', (-10.0, -5.0]),
          ('涨跌幅', (-5.0, -3.0]),  ('涨跌幅', (-3.0, -0.099]),
       ('涨跌幅', (-0.099, 0.099]),    ('涨跌幅', (0.099, 3.0]),
            ('涨跌幅', (3.0, 5.0]),     ('涨跌幅', (5.0, 10.0]),
          ('涨跌幅', (10.0, 20.0])],
      dtype='object')

In [95]:
# create a dictionary mapping the old column names to the new column names
column_name_mapping = dict(zip(result.columns[14:], x))

# use the rename method to rename the columns
df = df.rename(columns=column_name_mapping)


In [96]:
column_name_mapping


{('涨跌幅', Interval(-20.0, -10.0, closed='right')): '跌停',
 ('涨跌幅', Interval(-10.0, -5.0, closed='right')): '跌<-5%',
 ('涨跌幅', Interval(-5.0, -3.0, closed='right')): '-5<-3%',
 ('涨跌幅', Interval(-3.0, -0.099, closed='right')): '-1<3%',
 ('涨跌幅', Interval(-0.099, 0.099, closed='right')): '平盘',
 ('涨跌幅', Interval(0.099, 3.0, closed='right')): '1<3%',
 ('涨跌幅', Interval(3.0, 5.0, closed='right')): '3-5%',
 ('涨跌幅', Interval(5.0, 10.0, closed='right')): '5%-涨停',
 ('涨跌幅', Interval(10.0, 20.0, closed='right')): '涨停'}

In [110]:
result.columns

Index([                    '开盘',                     '收盘',
                           '最高',                     '最低',
                          '成交量',                    '成交额',
                           '振幅',                    '涨跌幅',
                          '涨跌额',                    '换手率',
                         '股票代码',                   '板块名称',
                          '总股本',                    '总市值',
        ('涨跌幅', (-20.0, -10.0]),   ('涨跌幅', (-10.0, -5.0]),
          ('涨跌幅', (-5.0, -3.0]),  ('涨跌幅', (-3.0, -0.099]),
       ('涨跌幅', (-0.099, 0.099]),    ('涨跌幅', (0.099, 3.0]),
            ('涨跌幅', (3.0, 5.0]),     ('涨跌幅', (5.0, 10.0]),
          ('涨跌幅', (10.0, 20.0])],
      dtype='object')

In [97]:
result.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 71152 entries, 2023-03-01 to 2023-03-20
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   开盘                      71152 non-null  float64
 1   收盘                      71152 non-null  float64
 2   最高                      71152 non-null  float64
 3   最低                      71152 non-null  float64
 4   成交量                     71152 non-null  int64  
 5   成交额                     71152 non-null  float64
 6   振幅                      71152 non-null  float64
 7   涨跌幅                     71152 non-null  float64
 8   涨跌额                     71152 non-null  float64
 9   换手率                     71152 non-null  float64
 10  股票代码                    71152 non-null  object 
 11  板块名称                    71152 non-null  object 
 12  总股本                     68019 non-null  float64
 13  总市值                     68019 non-null  float64
 14  (涨跌幅, (-20.0, -10.0])

In [108]:
mydf=pd.read_csv("result_20230322.csv")

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 71152 entries, 2023-03-01 to 2023-03-20
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   开盘      71152 non-null  float64
 1   收盘      71152 non-null  float64
 2   最高      71152 non-null  float64
 3   最低      71152 non-null  float64
 4   成交量     71152 non-null  int64  
 5   成交额     71152 non-null  float64
 6   振幅      71152 non-null  float64
 7   涨跌幅     71152 non-null  float64
 8   涨跌额     71152 non-null  float64
 9   换手率     71152 non-null  float64
 10  股票代码    71152 non-null  object 
 11  板块名称    71152 non-null  object 
 12  总股本     68019 non-null  float64
 13  跌停      68019 non-null  float64
dtypes: float64(11), int64(1), object(2)
memory usage: 10.2+ MB


In [106]:

# create a dictionary mapping the old column names to the new column names
column_name_mapping = dict(zip(mydf.columns[14:], x))

# use the rename method to rename the columns
df = df.rename(columns=column_name_mapping)
df

,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率,股票代码,板块名称,总股本,跌停
日期,,,,,,,,,,,,,,
2023-03-01,22.55,22.42,22.83,22.18,98735,221958533.0,2.90,0.18,0.04,3.03,300119,动物保健,468018786.0,1.049298e+10
2023-03-02,22.44,21.73,22.54,21.64,53425,117040504.0,4.01,-3.08,-0.69,1.64,300119,动物保健,468018786.0,1.017005e+10
2023-03-03,21.89,21.27,22.18,21.12,57008,122635566.0,4.88,-2.12,-0.46,1.75,300119,动物保健,468018786.0,9.954760e+09
2023-03-06,21.30,21.73,21.89,21.17,49037,106350983.0,3.39,2.16,0.46,1.50,300119,动物保健,468018786.0,1.017005e+10
2023-03-07,21.82,21.27,22.22,21.17,41433,89342602.0,4.83,-2.12,-0.46,1.27,300119,动物保健,468018786.0,9.954760e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-14,12.66,12.56,12.78,12.39,30798,38558589.0,3.07,-1.18,-0.15,0.63,603010,其他化学制品,589578593.0,7.405107e+09
2023-03-15,12.90,13.26,13.80,12.79,129459,173116219.0,8.04,5.57,0.70,2.67,603010,其他化学制品,589578593.0,7.817812e+09
2023-03-16,13.00,13.36,13.50,12.99,95162,126933949.0,3.85,0.75,0.10,1.96,603010,其他化学制品,589578593.0,7.876770e+09


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 71152 entries, 2023-03-01 to 2023-03-20
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   开盘      71152 non-null  float64
 1   收盘      71152 non-null  float64
 2   最高      71152 non-null  float64
 3   最低      71152 non-null  float64
 4   成交量     71152 non-null  int64  
 5   成交额     71152 non-null  float64
 6   振幅      71152 non-null  float64
 7   涨跌幅     71152 non-null  float64
 8   涨跌额     71152 non-null  float64
 9   换手率     71152 non-null  float64
 10  股票代码    71152 non-null  object 
 11  板块名称    71152 non-null  object 
 12  总股本     68019 non-null  float64
 13  跌停      68019 non-null  float64
dtypes: float64(11), int64(1), object(2)
memory usage: 10.2+ MB


In [113]:
import pandas as pd

# Create a DataFrame with a CategoricalIndex
data = {'A': [1, 2, 3], 'B': [4, 5, 6]}
df = pd.DataFrame(data, index=pd.CategoricalIndex(
    ['X', 'Y', 'Z'], categories=['Z', 'Y', 'X']))

# Print the original DataFrame
print(df)

# Rename the categories in the index
df.index = df.index.rename_categories(
    ['Category 3', 'Category 2', 'Category 1'])

# Print the updated DataFrame
print(df)


   A  B
X  1  4
Y  2  5
Z  3  6
            A  B
Category 1  1  4
Category 2  2  5
Category 3  3  6


In [131]:
result.columns[:14].to_list()


['开盘',
 '收盘',
 '最高',
 '最低',
 '成交量',
 '成交额',
 '振幅',
 '涨跌幅',
 '涨跌额',
 '换手率',
 '股票代码',
 '板块名称',
 '总股本',
 '总市值']

In [133]:
a=result.columns[:14].to_list()
a.extend(x)
a

['开盘',
 '收盘',
 '最高',
 '最低',
 '成交量',
 '成交额',
 '振幅',
 '涨跌幅',
 '涨跌额',
 '换手率',
 '股票代码',
 '板块名称',
 '总股本',
 '总市值',
 '跌停',
 '跌<-5%',
 '-5<-3%',
 '-1<3%',
 '平盘',
 '1<3%',
 '3-5%',
 '5%-涨停',
 '涨停']

In [125]:
result.columns =a

In [126]:
result.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 71152 entries, 2023-03-01 to 2023-03-20
Data columns (total 23 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   开盘      71152 non-null  float64
 1   收盘      71152 non-null  float64
 2   最高      71152 non-null  float64
 3   最低      71152 non-null  float64
 4   成交量     71152 non-null  int64  
 5   成交额     71152 non-null  float64
 6   振幅      71152 non-null  float64
 7   涨跌幅     71152 non-null  float64
 8   涨跌额     71152 non-null  float64
 9   换手率     71152 non-null  float64
 10  股票代码    71152 non-null  object 
 11  板块名称    71152 non-null  object 
 12  总股本     68019 non-null  float64
 13  总市值     68019 non-null  float64
 14  跌停      71152 non-null  int64  
 15  跌<-5%   71152 non-null  int64  
 16  -5<-3%  71152 non-null  int64  
 17  -1<3%   71152 non-null  int64  
 18  平盘      71152 non-null  int64  
 19  1<3%    71152 non-null  int64  
 20  3-5%    71152 non-null  int64  
 21  5%-涨停   71152 non-

In [122]:
a=[1,2]
b=[3,4]
a+b

[1, 2, 3, 4]

In [134]:
result.to_csv("result_20230323.csv")

In [135]:
df=pd.read_csv("result_20230323.csv",index_col=0,parse_dates=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 71152 entries, 2023-03-01 to 2023-03-20
Data columns (total 23 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   开盘      71152 non-null  float64
 1   收盘      71152 non-null  float64
 2   最高      71152 non-null  float64
 3   最低      71152 non-null  float64
 4   成交量     71152 non-null  int64  
 5   成交额     71152 non-null  float64
 6   振幅      71152 non-null  float64
 7   涨跌幅     71152 non-null  float64
 8   涨跌额     71152 non-null  float64
 9   换手率     71152 non-null  float64
 10  股票代码    71152 non-null  int64  
 11  板块名称    71152 non-null  object 
 12  总股本     68019 non-null  float64
 13  总市值     68019 non-null  float64
 14  跌停      71152 non-null  int64  
 15  跌<-5%   71152 non-null  int64  
 16  -5<-3%  71152 non-null  int64  
 17  -1<3%   71152 non-null  int64  
 18  平盘      71152 non-null  int64  
 19  1<3%    71152 non-null  int64  
 20  3-5%    71152 non-null  int64  
 21  5%-涨停   71152 non-